In [145]:
import pandas as pd
import openpyxl
import numpy as np
from openpyxl.styles import PatternFill
from openpyxl.styles import Font
from openpyxl.styles import numbers

##Functions
def autofit_columns(worksheet):
    for column in worksheet.columns:
        max_length = max(len(str(cell.value)) for cell in column)
        adjusted_width = (max_length + 2) * 1.0
        worksheet.column_dimensions[column[0].column_letter].width = adjusted_width
    print('Autofitting columns complete')

## Data Sources

Source = pd.read_excel('StaffListing_SalaryReview.xlsx')
PS_Source = pd.read_excel('PS_Input.xlsx')
Class = pd.read_excel('Class.xlsx', index_col=None, usecols="A:B")
Positions = pd.read_excel('Class.xlsx', index_col=None, usecols="F:G")
print('Import Data Sources Success')

##Dropping unneeded columns
PCS_1 = Source.drop(['Educator ID','Fed Funds'], axis=1)
print('Dropped unused columns')

##Removing Null Records
Class = Class.dropna(subset=['Class'])
print('Null records for class removed')

PCS_2 = PCS_1.dropna(subset=['SSN'])
print('Null records for PCS data removed')

##Updating PCS values based on PS designations
PCS_2 = PCS_2.merge(Class, how='left', left_on='Class', right_on='Class' )
PCS_2 = PCS_2.merge(Positions, how='left', left_on='Positions', right_on='Positions' )
print('Class and Positons ppdated')




##Renaming columns for PCS and PS data
PCS_source= PCS_2.rename(columns={'Cert Num': 'CertificateID',
                             'GRADE': 'PCS_Grade',
                             'Years Teach': 'Teaching_Exp', 
                             'FTE':'PCS_FTE',
                             'Code': 'Position_Code',
                              'Annual Sal': 'PCS_Salary'})

PS_Input= PS_Source.rename(columns={'GRADE': 'PS_Grade',
                                    'FTE': 'PS_FTE',
                                    'HCS_PCS_TOTSAL': 'PS_Salary',
                                    'PS_Step': 'Step'})
print('Columns Renamed')

#Formatting SSN and EMPLID
PCS_source['SSN'] = PCS_source['SSN'].apply('{:0>09}'.format)
PS_Input['NATIONAL_ID'] = PS_Input['NATIONAL_ID'].apply('{:0>09}'.format)
PS_Input['EMPLID'] = PS_Input['EMPLID'].apply('{:0>06}'.format)
print('SSN and EMPLID properly formatted')

##Merging PCS Data and PS Data into one file##
Merge = PCS_source.merge(PS_Input, how='left', left_on='SSN', right_on='NATIONAL_ID' )
print('Left outer join for PCS data and PS data complete')

##Making columns numeric for calculations
Merge['PCS_Grade'] = pd.to_numeric(Merge['PCS_Grade'])
Merge['PS_Grade'] = pd.to_numeric(Merge['PS_Grade'])
Merge['Teaching_Exp'] = pd.to_numeric(Merge['Teaching_Exp'])
Merge['Step'] = pd.to_numeric(Merge['Step'])
Merge['PCS_Salary'] = pd.to_numeric(Merge['PCS_Salary'])
Merge['PS_Salary'] = pd.to_numeric(Merge['PS_Salary'])
print('Fields Ready for Calculaitons')

##Replaced Null vallues with 0
Merge['CertificateID'].fillna("000000", inplace = True)
Merge['PCS_Grade'].fillna(0, inplace = True)
Merge['PS_Grade'].fillna(0, inplace = True)
Merge['Teaching_Exp'].fillna(0, inplace = True)
Merge['Step'].fillna(0, inplace = True)
Merge['PS_Salary'] = Merge['PS_Salary'].replace(np.nan, 0)
print('Null Values Replaced')

##Creating the compare columns
Merge['StepDiff'] = Merge['Teaching_Exp'] - Merge['Step']
Merge['PayDiff'] = Merge['PCS_Salary'] - Merge['PS_Salary'] 
Merge['GradDiff']= Merge['PCS_Grade'] - Merge['PS_Grade']
print('Grade, Step and Pay columns compared')


##Formatting for currency
# Merge['PCS_Salary'] = Merge['PCS_Salary'].map(lambda x: f"{x:.2f}")
# Merge['PS_Salary'] = Merge['PS_Salary'].map(lambda x: f"{x:.2f}")

##Updating Dataframe to take only needed columns
Merge = Merge[['SSN','CertificateID','Name','PCS_Grade','PS_Grade','GradDiff','Teaching_Exp','Step','StepDiff',
              'PCS_Salary','PS_Salary','PayDiff','LOCATION','PAYGROUP','PCS_FTE','PS_FTE','Position_Code','EMPLID',
              'EMPL_STATUS']]

##Creating the output
Merge.to_excel('PCS_Salary_Compare.xlsx', index=False)

##Color Formatting
wb = openpyxl.load_workbook(filename="PCS_Salary_Compare.xlsx")
ws = wb['Sheet1']

autofit_columns(ws)


yellow = PatternFill(fill_type='solid',start_color ='F2E399')
orange= PatternFill(fill_type='solid',start_color ='FFB641')
red = PatternFill(fill_type='solid',start_color ='FF0000')

font_style = Font(name='Calibri',size=11)
ws.style = font_style

currency = '"$"#,##0.00_-'
for column in ws['B']:
    if column.value == '000000':
        column.fill = yellow
        
for column in ws['F']:
    if column.value != 0:
        column.fill = orange
        
for column in ws['I']:
    if column.value != 0:
        column.fill = orange
        
for column in ws['L']:
    if column.value != 0.00:
        column.fill = red
    column.number_format = currency
for column in ws['J']:
    column.number_format = currency
for column in ws['K']:
    column.number_format = currency

wb.save("PCS_Salary_Compare.xlsx")
wb.close()

print('Color and Style formatting complete') 


# print(PCS_source.head())
# print(PS_Input.head())
# print(dir(openpyxl))
# print(help(openpyxl.worksheet.dimensions))
print([Output.shape[0]], ' Records Processed')
print("Process Complete")

Import Data Sources Success
Dropped unused columns
Null records for class removed
Null records for PCS data removed
Class and Positons ppdated
Columns Renamed
SSN and EMPLID properly formatted
Left outer join for PCS data and PS data complete
Fields Ready for Calculaitons
Null Values Replaced
Grade, Step and Pay columns compared
Autofitting columns complete
Color and Style formatting complete
[3685]  Records Processed
Process Complete
